In [17]:
from dotenv import load_dotenv
load_dotenv(".env")
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec


In [14]:
pinecone_api_key = os.getenv("PINECONE_API_KEY")
print(f"Pinecone API Key: {pinecone_api_key}")
pc = Pinecone(api_key=pinecone_api_key)
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1") 
)

Pinecone API Key: 5c73f20b-8ed8-46e9-bda1-31ef842b9c0a


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'c9b0a550728e3bbc3b7df2fbe888b745', 'Date': 'Mon, 19 Aug 2024 16:29:25 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [15]:
import json 
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Adams',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Excellent professor who makes complex topics easy to understand.'},
 {'professor': 'Dr. John Miller',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Great lectures, but the exams are challenging.'},
 {'professor': 'Dr. Sarah Thompson',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Knowledgeable but not very engaging in class.'},
 {'professor': 'Dr. Robert Brown',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Very approachable and helpful outside of class.'},
 {'professor': 'Dr. Lisa Clark',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Interesting lectures, but sometimes the material is too dense.'},
 {'professor': 'Dr. James Wilson',
  'subject': 'History',
  'stars': 2,
  'review': 'The lectures are dull and the readings are overwhelming.'},
 {'professor': 'Dr. Angela Davis',
  'subject': 'Literature',
  'stars': 5,
  'review': 'Passionate about the subject and makes

In [18]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding 
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata":{
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [20]:
index = pc.Index('rag')
index.upsert(
    vectors = processed_data,
    namespace="ns1",
    
)

{'upserted_count': 20}